In [1]:
import tensorflow as tf
import dlib
import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (96,96))
    # Scale image to be between 0 and 1 
    img = img / 255.0
    
    # Return image
    return img

In [4]:
# Using TFLIte without OPTIMIZE
interpreter = tf.lite.Interpreter(model_path="a_net.tflite")
interpreter.allocate_tensors()

In [5]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [6]:
input_details

[{'name': 'serving_default_validation_image:0',
  'index': 0,
  'shape': array([ 1, 96, 96,  3], dtype=int32),
  'shape_signature': array([-1, 96, 96,  3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'serving_default_input_image:0',
  'index': 1,
  'shape': array([ 1, 96, 96,  3], dtype=int32),
  'shape_signature': array([-1, 96, 96,  3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [7]:
output_details

[{'name': 'StatefulPartitionedCall:0',
  'index': 38,
  'shape': array([1, 1], dtype=int32),
  'shape_signature': array([-1,  1], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [7]:
import glob

database = glob.glob("database/*")
print(database)

['database/mohsin', 'database/ehsan', 'database/arham']


In [8]:
label_map = {'arham': 0, 'ehsan': 2, 'mohsin': 1}
label_map

{'arham': 0, 'ehsan': 2, 'mohsin': 1}

In [14]:
# Arham Image
# input_img = preprocess(os.path.join("dataset", "cropped", "arham", "9a1b3c24-99df-11ec-ab0e-842b2bb3a5b6.jpg"))
# input_img = preprocess(os.path.join("dataset", "cropped", "mohsin", "0a2bbaa0-99e0-11ec-826f-842b2bb3a5b6.jpg"))
input_img = preprocess(os.path.join("dataset", "cropped", "ehsan", "4a02d925-99e0-11ec-be8c-842b2bb3a5b6.jpg"))

In [95]:
input_img.shape

TensorShape([96, 96, 3])

In [102]:
validation_mappings = []

In [103]:
for database_dir in database:
    label = database_dir.split("/")[-1]
    for validate_img in glob.glob(database_dir+"/*"):
        print(validate_img)
        validation_img = preprocess(validate_img)
        validation_mappings.append([validation_img, label_map[label]])

database/mohsin/0a3cd26f-99e0-11ec-b772-842b2bb3a5b6.jpg
database/ehsan/4a0dbcc7-99e0-11ec-aa89-842b2bb3a5b6.jpg
database/arham/9a0cac9e-99df-11ec-8d31-842b2bb3a5b6.jpg


In [105]:
validation_mappings = np.array(validation_mappings)
validation_mappings.shape

(3, 2)

In [106]:
np.save("embeddings", validation_mappings)

In [10]:
vp = np.load("embeddings.npy", allow_pickle=True)

In [12]:
import time

In [15]:
start = time.time()
results = [[], [], []]
for validate in vp:
    embs, label = validate
    interpreter.set_tensor(input_details[0]['index'], np.expand_dims(input_img, axis=0))
    interpreter.set_tensor(input_details[1]['index'], np.expand_dims(embs, axis=0))
    interpreter.invoke()
    pred = interpreter.get_tensor(output_details[0]['index'])
    if label == 0:
        results[0].append(pred)
    if label == 1:
        results[1].append(pred)
    if label == 2:
        results[2].append(pred)
print(f"Total Seconds: {round(time.time()-start, 2)}sec.")

Total Seconds: 1.62sec.


In [16]:
if np.argmax(results) == 0:
    print("Arham")
if np.argmax(results) == 1:
    print("Mohsin")
if np.argmax(results) == 2:
    print("Ehsan")

Ehsan
